In [81]:
import pandas as pd
import numpy as np
import scipy
import sklearn 
import datetime as dt
import xgboost as XGB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel


from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV
from sklearn.utils import shuffle

from sklearn import metrics
from sklearn.model_selection import train_test_split
pd.set_option('display.float_format', lambda x: '%.3f' % x)


# pd.options.display.float_format = '{:20,.0f}'.format # suprimimos la notacion cientifica en los outputs

In [82]:
path = 'data/'
features = pd.read_csv(path + 'features.csv')
test = pd.read_csv(path + 'testModified.csv')

In [83]:
x = features.drop('precio',1)
y = features['precio']
tst_sz = 0.3 # Mejor valor
seed = 139
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=tst_sz,random_state=seed)
x, y = shuffle(x, y, random_state=seed)

In [84]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] # Numero de arboles
max_features = ['auto', 'sqrt'] # Numero de features a considerar en cada split
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # Cantidad maxima de niveles en el arbol
max_depth.append(None)
min_samples_split = [2, 5, 10] # Numero minimo de muestras para splitear nodo
min_samples_leaf = [1, 2, 4] # Numero minimo de muestras requeridas en cada nodo hoja
bootstrap = [True, False] # Metodo seleccion muestras para entrenar cada arbol

In [85]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=100, random_state=42, n_jobs = -1)
rf_random.fit(x, y)

scoring = {'abs_error': 'neg_mean_absolute_error'}
scores = cross_validate(rf_random.best_estimator_, x, y, cv=10, scoring=scoring, return_train_score=True, verbose=30, n_jobs=-1)
print("MAE :", abs(scores['test_abs_error'].mean()))

sel = SelectFromModel(RandomForestRegressor(n_estimators=200, max_features='auto', max_depth=100, min_samples_split=5, min_samples_leaf=3, bootstrap=True))
sel.fit(x, y)

x.columns[sel.get_support()]

In [86]:
rf_model = RandomForestRegressor(n_estimators=200, max_features='auto', max_depth=100, min_samples_split=5, min_samples_leaf=3, bootstrap=True)

In [87]:
scoring = {'abs_error': 'neg_mean_absolute_error'}
scores = cross_validate(rf_model, x, y, cv=4, scoring=scoring, return_train_score=True, verbose=30, n_jobs=2)
print("MAE :", abs(scores['test_abs_error'].mean()))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:  4.4min
[Parallel(n_jobs=2)]: Done   2 out of   4 | elapsed:  4.4min remaining:  4.4min


MAE : 657027.9793820439


[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:  9.8min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:  9.8min finished


In [88]:
rf_model.fit(x,y)
rf_predict = rf_model.predict(test.drop(['id'], axis=1))

In [89]:
res = pd.DataFrame(rf_predict, index=test.id, columns=['precio'])

In [90]:
res = res.rename({'precio':'target'}, axis=1)

In [91]:
res.to_csv("predictions/rf_predictions.csv", header = True)

In [92]:
res.shape

(60000, 1)

In [93]:
res

,target
id,
4941,5306295.699
51775,1099183.366
115253,2329376.034
299321,1464009.519
173570,744980.206
30862,1792473.705
244471,2137816.298
127794,548334.860
71558,2613294.451
